In [1]:
import numpy as np
from quaos.paulis import PauliSum, PauliString
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator
import itertools
from itertools import combinations
import time

In [19]:
def group_indices(lst):
    """
    Groups indices of the same value in a list into sublists.
    For example, if the input list is [1, 2, 1, 3, 2], the output will be [[0, 2], [1, 4], [3]].
    """
    index_dict = defaultdict(list)
    for idx, value in enumerate(lst):
        index_dict[value].append(idx)
    
    return [indices for indices in index_dict.values()]

def SWAP(P,q0,q1):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(q0,q1,2),CX(q1,q0,2),CX(q0,q1,2)])
    P = C.act(P)
    return(P)

def SWAP_Symmetric_PauliSum(n_paulis,n_qubits):
    # Step 1: Create coefficients, possibly with bands of the same value
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    # Step 2: Assign random Paulis to first qubit
    pauli_strings = ['' for i in range(n_paulis)]
    first_qubit_paulis = []
    for i in range(n_paulis):
        pauli_choice = np.random.randint(0,4)
        first_qubit_paulis.append(pauli_choice)
        if pauli_choice == 0:
            pauli_strings[i] += 'x0z0 '
        elif pauli_choice == 1:
            pauli_strings[i] += 'x1z0 '
        elif pauli_choice == 2:
            pauli_strings[i] += 'x0z1 '
        elif pauli_choice == 3:
            pauli_strings[i] += 'x1z1 '
    first_qubit_paulis = np.array(first_qubit_paulis)

    # Step 3: Create SWAP symmetric counter part
    # First the ones with asymmetry "I x Z, Z x I" etc
    second_qubit_paulis = np.copy(first_qubit_paulis)
    asym_pairs = []
    for b in sym_bands:
        band_length = len(b)
        band_indexes = b.copy()
        #first_qubit_paulis_b = first_qubit_paulis[b]
        max_swaps = np.floor(band_length/2)
        n_swaps = np.random.randint(0,max_swaps+1)
        for i in range(n_swaps):
            swap_index_1 = np.random.choice(band_indexes)
            band_indexes.remove(swap_index_1)
            swap_index_2 = np.random.choice(band_indexes)
            band_indexes.remove(swap_index_2)
            if first_qubit_paulis[swap_index_2] != first_qubit_paulis[swap_index_1]:
                asym_pairs.append((swap_index_1,swap_index_2))
            second_qubit_paulis[swap_index_1] = first_qubit_paulis[swap_index_2]
            second_qubit_paulis[swap_index_2] = first_qubit_paulis[swap_index_1]
        print()
    print(asym_pairs)
    
    # now fill in all at once
    for i in range(n_paulis):
        pauli_choice = second_qubit_paulis[i]
        if pauli_choice == 0:
            pauli_strings[i] += 'x0z0 '
        elif pauli_choice == 1:
            pauli_strings[i] += 'x1z0 '
        elif pauli_choice == 2:
            pauli_strings[i] += 'x0z1 '
        elif pauli_choice == 3:
            pauli_strings[i] += 'x1z1 '

    # Step 4: Add the additional qubits to fill up the Hamiltonian
    # first the ones with the asymmetry swap, as they need to have the same Paulis afterwards
    q_dims = [2 for i in range(2*(n_qubits-2))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for ap in asym_pairs:
        api = ap[0]
        apj = ap[1]
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-2):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[api] += f"x{r}z{s} "
            pauli_strings[apj] += f"x{r}z{s} "

        pauli_strings[api].strip()
        pauli_strings[apj].strip()

    # Step 5: Add the remaining Paulis
    for i in range(n_paulis):
        if i not in [ap[0] for ap in asym_pairs] and i not in [ap[1] for ap in asym_pairs]:
            pauli_index = np.random.choice(available_paulis)
            available_paulis.remove(pauli_index)
            exponents = int_to_bases(pauli_index, q_dims)
            for j in range(n_qubits-2):
                r, s = int(exponents[2*j]), int(exponents[2*j+1])
                pauli_strings[i] += f"x{r}z{s} "
            pauli_strings[i].strip()
    
    # Step 6: Create the PauliSum object
    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)

    # Step 7: Randomize via random circuits
    n_qubits = P.n_qudits()
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits 
    '''
    for i in range(10):
        a = np.random.randint(n_qubits)
        b = np.random.randint(n_qubits)
        if a != b:
            P = SWAP(P,a,b)
    '''
    return(P)



In [25]:
P = SWAP_Symmetric_PauliSum(20,5)
print(P)






[(7, 8), (9, 10), (16, 15), (12, 13)]
(0.24545759821474206+0j) |x1z0 x1z1 x1z0 x1z0 x1z1 | 0 
(0.3856591517984129+0j)  |x0z0 x1z1 x0z1 x0z0 x1z0 | 0 
(-0.3856591517984129+0j) |x1z1 x0z1 x1z0 x0z1 x0z0 | 0 
(-1.45699321953928+0j)   |x0z0 x0z1 x0z0 x1z0 x1z1 | 0 
(1.45699321953928-0j)    |x1z1 x1z1 x0z0 x0z0 x0z1 | 0 
(-1.45699321953928+0j)   |x0z0 x0z1 x0z0 x1z0 x0z1 | 0 
(-2.04931480971289+0j)   |x1z0 x1z1 x1z0 x0z0 x1z0 | 0 
(0.6855491988530842-0j)  |x0z0 x0z0 x0z1 x0z1 x1z0 | 0 
(-0.6855491988530842+0j) |x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.6583772067296557+0j) |x1z1 x0z1 x1z0 x1z1 x0z0 | 0 
(-0.6583772067296557+0j) |x0z0 x0z1 x1z1 x0z1 x0z0 | 0 
(-0.982981724509194+0j)  |x1z1 x0z0 x1z1 x0z1 x0z0 | 0 
(-0.09360197303421115+0j)|x1z1 x0z0 x0z0 x1z0 x0z1 | 0 
(-0.09360197303421115+0j)|x0z0 x0z0 x0z1 x0z0 x0z1 | 0 
(0.09360197303421115-0j) |x1z1 x1z0 x1z1 x0z1 x0z0 | 0 
(-0.09360197303421115+0j)|x1z1 x1z1 x1z0 x1z0 x1z0 | 0 
(0.09360197303421115-0j) |x0z1 x1z1 x1z0 x0z1 x0z0 | 0 
(0.34

In [32]:
def number_of_SUM_X(r_control, r_target, d):
    """
    Find the smallest positive integer N such that:
        (r_target + N * r_control) % d == 0

    This counts the number N of SUM gates needed to cancel out the X part of a Pauli operator.
    """
    N = 1
    while (r_target + N * r_control) % d != 0:
        if N > d:
            raise Exception('Error in Exponents r_control = ' + str(r_control) + ' r_target = ' + str(r_target))
        N += 1
        
    return N


def number_of_SUM_Z(s_control, s_target, d):
    N = 1
    while (s_control - N * s_target) % d != 0:
        if N > d:
            raise Exception('Error in Exponents s_control = ' + str(s_control) + ' s_target = ' + str(s_target))
        N += 1
        
    return N


def number_of_S(x_exp, z_exp, d):
    N = 1
    while (x_exp * N + z_exp) % d != 0:
        if N > d:
            raise Exception('Error in Exponents x_exp = ' + str(x_exp) + ' z_exp = ' + str(z_exp))
        N += 1
        
    return N


def cancel_X(pauli_sum, qudit, pauli_index, C, q_max):
    list_of_gates = []
    for i in range(qudit + 1, q_max):
        if pauli_sum.x_exp[pauli_index, i]:
            list_of_gates += [CX(qudit, i, pauli_sum.dimensions[qudit])] * number_of_SUM_X(pauli_sum.x_exp[pauli_index, qudit],
                                                                                           pauli_sum.x_exp[pauli_index, i],
                                                                                           pauli_sum.dimensions[i])
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C


def cancel_Z(pauli_sum, qudit, pauli_index, C, q_max):

    list_of_gates = []
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    for i in range(qudit + 1, q_max):
        if pauli_sum.z_exp[pauli_index, i]:
            list_of_gates += [CX(i, qudit, pauli_sum.dimensions[qudit])] * number_of_SUM_Z(pauli_sum.z_exp[pauli_index, i],
                                                                                           pauli_sum.x_exp[pauli_index, qudit],
                                                                                           pauli_sum.dimensions[i])
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C


def cancel_Y(pauli_sum, qudit, pauli_index, C):
    list_of_gates = [S(qudit, pauli_sum.dimensions[qudit])] * number_of_S(pauli_sum.x_exp[pauli_index, qudit],
                                                                          pauli_sum.z_exp[pauli_index, qudit],
                                                                          pauli_sum.dimensions[qudit])
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C


def cancel_pauli(P, current_qudit, pauli_index, circuit, n_q_max):
    """
    Needs an x component on current_qudit

    P -> p_1 ... p_current_qudit  I I ... I p_n_q_max p.... p_n_paulis
    
    """
    # add CX gates to cancel out all non-zero X-parts on Pauli pauli_index, i > qudit
    if any(P.x_exp[pauli_index, i] for i in range(current_qudit + 1, n_q_max)):
        P, circuit = cancel_X(P, current_qudit, pauli_index, circuit, n_q_max)

    # add CZ gates to cancel out all non-zero Z-parts on Pauli pauli_index, i > qudit
    if any(P.z_exp[pauli_index, i] for i in range(current_qudit + 1, n_q_max)):
        P, circuit = cancel_Z(P, current_qudit, pauli_index, circuit, n_q_max)

    # if indexed Pauli, qudit is Y, add S gate to make it X
    if P.z_exp[pauli_index, current_qudit] and P.x_exp[pauli_index, current_qudit]:
        P, circuit = cancel_Y(P, current_qudit, pauli_index, circuit)
    return P, circuit


def symplectic_reduction_iter_qudit_(P, C, pivots, current_qudit):
    n_p, n_q = P.n_paulis(), P.n_qudits()
    P = C.act(P)
    n_q_max = n_q
    # find n_q_max, the last qudit of the same dimension as current_qudit
    for i in range(n_q - current_qudit):
        if P.dimensions[current_qudit + i] != P.dimensions[current_qudit]:
            n_q_max = current_qudit + i - 1
            break

    # does the current qudit have any X or Z components?
    if any(P.x_exp[:, current_qudit]) or any(P.z_exp[:, current_qudit]):
        if not any(P.x_exp[:, current_qudit]):  # If it is z we need to add a Hadamard gate to make it an X
            g = H(current_qudit, P.dimensions[current_qudit])
            C.add_gate(g)
            P = g.act(P)

        current_pauli = min(i for i in range(n_p) if P.x_exp[i, current_qudit])  # first Pauli that has an x-component
        pivots.append((current_pauli, current_qudit, 'X'))

        P, C = cancel_pauli(P, current_qudit, current_pauli, C, n_q_max)

    # If there was previously a y we need to cancel the left over z parts
    if any(P.z_exp[:, current_qudit]):
        current_pauli = min(i for i in range(n_p) if P.z_exp[i, current_qudit])  # first Pauli that has a z-component
        pivots.append((current_pauli, current_qudit, 'Z'))

        g = H(current_qudit, P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)

        P, C = cancel_pauli(P, current_qudit, current_pauli, C, n_q_max)

        g = H(current_qudit, P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)
    
    return C, pivots

def reverse_cancel_X(pauli_sum, qudit, pauli_index, C):
    list_of_gates = []
    for i in range(qudit):
        if pauli_sum.x_exp[pauli_index, i]:
            list_of_gates += [CX(qudit, i, pauli_sum.dimensions[qudit])] 
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C


def reverse_cancel_Z(pauli_sum, qudit, pauli_index, C):
    list_of_gates = []
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    for i in range(qudit):
        if pauli_sum.z_exp[pauli_index, i]:
            list_of_gates += [CX(i, qudit, pauli_sum.dimensions[qudit])] 
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C


def reverse_cancel_Y(pauli_sum, qudit, pauli_index, C):
    list_of_gates = [S(qudit, pauli_sum.dimensions[qudit])] 
    C.add_gate(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return pauli_sum, C

def reverse_cancel_pauli(P, current_qudit, pauli_index, circuit):
    """
    Needs an x component on current_qudit
    """
    # add CX gates to cancel out all non-zero X-parts on Pauli pauli_index, i > qudit
    if any(P.x_exp[pauli_index, i] for i in range(current_qudit)):
        P, circuit = reverse_cancel_X(P, current_qudit, pauli_index, circuit)

    # add CZ gates to cancel out all non-zero Z-parts on Pauli pauli_index, i > qudit
    if any(P.z_exp[pauli_index, i] for i in range(current_qudit)):
        P, circuit = reverse_cancel_Z(P, current_qudit, pauli_index, circuit)

    # if indexed Pauli, qudit is Y, add S gate to make it X
    if P.z_exp[pauli_index, current_qudit] and P.x_exp[pauli_index, current_qudit]:
        P, circuit = reverse_cancel_Y(P, current_qudit, pauli_index, circuit)
    return P, circuit


def reverse_symplectic_reduction_iter_qudit_(P, C, pivots, current_qudit):
    n_p, n_q = P.n_paulis(), P.n_qudits()
    P = C.act(P)
    
    if any(P.x_exp[:, current_qudit]) or any(P.z_exp[:, current_qudit]):
        if not any(P.x_exp[:, current_qudit]):  # If it is z we need to add a Hadamard gate to make it an X
            g = H(current_qudit, P.dimensions[current_qudit])
            C.add_gate(g)
            P = g.act(P)

        current_pauli = min(i for i in range(n_p) if P.x_exp[i, current_qudit])  # first Pauli that has an x-component
        pivots.append((current_pauli, current_qudit, 'X'))

        P, C = reverse_cancel_pauli(P, current_qudit, current_pauli, C)
    
    if any(P.z_exp[:, current_qudit]):
        current_pauli = min(i for i in range(n_p) if P.z_exp[i, current_qudit])  # first Pauli that has a z-component
        pivots.append((current_pauli, current_qudit, 'Z'))

        g = H(current_qudit, P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)

        P, C = reverse_cancel_pauli(P, current_qudit, current_pauli, C)

        g = H(current_qudit, P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)
    
    return C, pivots
    

In [37]:

P1 = P.copy()
C = Circuit(dimensions=[2 for i in range(P.n_qudits())])

C,pivots = reverse_symplectic_reduction_iter_qudit_(P.copy(), C, [], 4)
P1 = C.act(P)
print(P1)
C,pivots = reverse_symplectic_reduction_iter_qudit_(P.copy(), C, [], 3)
P1 = C.act(P)
print(P1)
C,pivots = reverse_symplectic_reduction_iter_qudit_(P.copy(), C, [], 2)
P1 = C.act(P)
print(P1)

(0.24545759821474206+0j) |x0z0 x0z0 x0z0 x0z0 x1z0 | 1 
(0.3856591517984129+0j)  |x1z0 x0z0 x1z1 x1z0 x0z0 | 0 
(-0.3856591517984129+0j) |x0z0 x0z0 x0z0 x0z0 x0z1 | 1 
(-1.45699321953928+0j)   |x0z1 x1z1 x0z0 x0z1 x1z1 | 1 
(1.45699321953928-0j)    |x1z1 x1z1 x0z0 x0z0 x1z0 | 0 
(-1.45699321953928+0j)   |x0z0 x0z1 x0z0 x1z0 x1z0 | 0 
(-2.04931480971289+0j)   |x1z1 x0z1 x1z0 x1z1 x0z1 | 0 
(0.6855491988530842-0j)  |x0z1 x1z0 x0z1 x1z0 x1z1 | 1 
(-0.6855491988530842+0j) |x1z0 x0z1 x1z0 x0z1 x1z1 | 0 
(-0.6583772067296557+0j) |x0z0 x0z0 x0z0 x1z0 x1z1 | 0 
(-0.6583772067296557+0j) |x1z1 x0z0 x0z1 x0z0 x1z1 | 0 
(-0.982981724509194+0j)  |x0z0 x0z1 x0z1 x0z0 x1z1 | 1 
(-0.09360197303421115+0j)|x1z1 x0z0 x0z0 x1z0 x1z0 | 0 
(-0.09360197303421115+0j)|x0z0 x0z0 x0z1 x0z0 x1z0 | 0 
(0.09360197303421115-0j) |x1z1 x1z0 x1z1 x0z1 x0z0 | 0 
(-0.09360197303421115+0j)|x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(0.09360197303421115-0j) |x0z1 x1z1 x1z0 x0z1 x0z0 | 0 
(0.34273523568946446-0j) |x0z0 x0z0 x1z0 x0z1 x0